In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

Importing the libraries to be used.

In [2]:
from keras.datasets import mnist

Using keras only to import the dataset.

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
train_X_upd = np.zeros(784*train_X.shape[0]).reshape(train_X.shape[0], 784)
for i in range(train_X.shape[0]):
  train_X_upd[i] = train_X[i].reshape(784)
  train_X_upd[i] = train_X_upd[i]/256

print(type(train_X_upd[1]))
print(train_X_upd[1].size)

<class 'numpy.ndarray'>
784


Process the data to convert it from a 28x28 array to a 784x1 array. Further, the input is normalised by dividing the values by 256, so that the input layer activations lie between 0 and 1. 

In [5]:
def output(x):
  out = np.zeros(10)
  out[x] = 1.0
  return out

w = output(train_y[0])
w

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [6]:
output_arr = np.zeros(10*train_y.shape[0])
output_arr = output_arr.reshape(train_y.shape[0], 10)
output_arr.shape

for i in range(train_y.shape[0]):
  output_arr[i] = output(train_y[i])

output_arr
train_y = output_arr.copy()

Convert the labels into an array of size 10x1 which can be directly compared with the output layer activations.

In [7]:
weights_1 = np.zeros(20*784)
weights_2 = np.zeros(20*10)
weights_1 = weights_1.reshape(20, 784)
weights_2 = weights_2.reshape(10, 20)

Initialise the matrices storing the weights and biases.

In [8]:
train_y.shape

(60000, 10)

In [9]:
weights_1 = np.random.normal(size = (20, 784))
#weights_1 = weights_1/1000
#print(weights_1)
print(weights_1.shape)
weights_2 = np.random.normal(size = (10, 20))
#weights_2 = weights_2/1000
#print(weights_2)
print(weights_2.shape)

(20, 784)
(10, 20)


In [10]:
biases_1 = np.random.normal(size = 20)
print(biases_1)
print(biases_1.shape)
biases_2 = np.random.normal(size = 10)
print(biases_2)
print(biases_2.shape)

[ 1.22064634  1.20750232  1.3322664  -1.25315217  0.47681739  1.41691943
  0.77461365  0.21373472 -1.43695739  0.50442407  3.15298022 -1.59641728
  0.34782812 -0.26568961  0.05641448  0.3423221   0.36887472  0.50624228
 -1.73609729 -1.18555135]
(20,)
[-0.15857874  1.10055986  0.60768007  0.7620109  -0.79331765  0.84428034
 -0.13694238  1.21508766  0.25607097  0.44114827]
(10,)


Initialise the weights and biases with values from the normal distribution.

In [11]:
hiddenlayer_1 = np.zeros(20)
hl_1_z = np.zeros(20)

In [12]:
inputlayer = np.zeros(784)

In [13]:
outputlayer = np.zeros(10)
ol_z = np.zeros(10)

Create the layers of the neural net.

In [14]:
def sigmoid_num(x):
  if(x > 20):
    return 1
  if(x < -20):
    return 0
  return (1/(1 + np.exp(-x)))

sigmoid = np.vectorize(sigmoid_num)
sigmoid(-5)

array(0.00669285)

The sigmoid function is the activation function. It has been vectorised to make it easy to apply on numpy arrays. Also to avoid np.exp overflow issues, sigmoid has been defines specifically for |x|>20.

In [15]:
def feed(x):
  global hiddenlayer_1
  global outputlayer
  global hl_1_z
  global ol_z
  np.matmul(weights_1, x, out = hl_1_z)
  hl_1_z += biases_1
  hiddenlayer_1 = sigmoid(hl_1_z)
  np.matmul(weights_2, hiddenlayer_1, out = ol_z)
  ol_z += biases_2
  outputlayer = sigmoid(ol_z)
  
feed(train_X_upd[1])
print(hl_1_z)
print(outputlayer)

[  3.24771762  -6.37457949   4.87170967   8.3734047   17.51443475
  -7.7204843   -0.34139499 -17.23148877 -14.2516053   -1.85417363
   2.24104325   9.88785427  11.97166063  -9.48347363   3.63546879
  17.98470011   2.18114539  -0.62516614  17.28374663  14.16825928]
[0.65294879 0.99718024 0.54623235 0.02367822 0.07918248 0.64149949
 0.94420398 0.45835695 0.00163452 0.97831872]


Function to obtain the output for a given input.

In [16]:
def cost_error(res):
  return np.dot((outputlayer - res), (outputlayer - res))

In [17]:
def cost(sample, sample_y):
  c = 0.0
  for i in range(sample.shape[0]):
    c += cost_error(sample_y[i])
  c/=(2*sample.shape[0])
  return c

a = cost(train_X_upd[1].reshape(1,784), train_y[1].reshape(1,10))
print(a)

1.945128657746301


Cost function - returns the quadratic function $x^2/2$. 

In [18]:
def sigmoid_prime_num(x):
  return ((sigmoid_num(x))*(1 - sigmoid_num(x)))

sigmoid_prime = np.vectorize(sigmoid_prime_num)

error_output = np.zeros(10)
error_hl_1 = np.zeros(20)
temp = np.zeros(20)
def backpropagate(res):
  global error_output
  global error_hl_1
  global outputlayer
  global temp
  np.multiply(outputlayer - res, sigmoid_prime(ol_z), out = error_output)
  np.matmul(np.transpose(weights_2), error_output, out = temp)
  np.multiply(temp, sigmoid_prime(hl_1_z), out = error_hl_1)

backpropagate(train_y[1])

sigmoid_prime is the derivative of the sigmoid function.

In [19]:
del_w_1 = np.zeros(20*784).reshape(20,784)
del_w_2 = np.zeros(10*20).reshape(10,20)

del_bias_1 = np.zeros(20)
del_bias_2 = np.zeros(10)

def create_del():
  global del_w_1
  global del_w_2
  global del_bias_1
  global del_bias_2
  np.outer(error_hl_1, np.transpose(inputlayer.reshape(784,1)), out = del_w_1)
  np.outer(error_output, np.transpose(hiddenlayer_1), out = del_w_2)
  del_bias_1 = error_hl_1.copy()
  del_bias_2 = error_output.copy()

create_del()
print(del_w_1)
print(del_w_2)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [-0. -0. -0. ... -0. -0. -0.]
 ...
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
[[-7.57021229e-02 -1.33807978e-04 -7.80462653e-02 -7.86259629e-02
  -7.86441179e-02 -3.48746533e-05 -3.26743241e-02 -2.58300801e-09
  -5.08478315e-08 -1.06471248e-02 -7.10844648e-02 -7.86401259e-02
  -7.86436227e-02 -5.98429427e-06 -7.66234395e-02 -7.86441186e-02
  -7.06651656e-02 -2.74159227e-02 -7.86441174e-02 -7.86440646e-02]
 [ 2.69899264e-03  4.77062904e-06  2.78256788e-03  2.80323572e-03
   2.80388300e-03  1.24337904e-06  1.16493114e-03  9.20914676e-11
   1.81286756e-09  3.79599811e-04  2.53436020e-03  2.80374067e-03
   2.80386534e-03  2.13356846e-07  2.73184015e-03  2.80388302e-03
   2.51941101e-03  9.77454406e-04  2.80388298e-03  2.80388109e-03]
 [ 1.30325729e-01  2.30358431e-04  1.34361311e-01  1.35359295e-01
   1.35390550e-01  6.00388002e-05  5.62508020e-02  4.44680268e-09
   8.75375812e

Function to calculate the entries of the gradient of the cost function.

In [20]:
learnrate = 2.0

def learn():
  global learnrate
  global weights_1
  global weights_2
  global biases_1
  global biases_2
  train_X_upd_s = train_X_upd.copy()
  train_y_s = train_y.copy()

  indices = np.arange(train_X_upd_s.shape[0])
  np.random.shuffle(indices)
  np.random.shuffle(indices)
  train_y_s = train_y_s[indices]
  train_X_upd_s = train_X_upd_s[indices]

  for i in range(6000):
    del_w_1_avg = np.zeros(20*784).reshape(20,784)
    del_w_2_avg = np.zeros(10*20).reshape(10,20)

    del_bias_1_avg = np.zeros(20)
    del_bias_2_avg = np.zeros(10)
    avg_cost = 0.0
    for j in range(10):
      feed(train_X_upd[i*10+j])
      avg_cost += cost(train_X_upd_s[i*10+j].reshape(1,784), train_y_s[i*10+j].reshape(10,1))
      backpropagate(train_y[i*10+j])
      create_del()
      del_w_1_avg += del_w_1
      del_w_2_avg += del_w_2
      del_bias_1_avg += del_bias_1
      del_bias_2_avg += del_bias_2
    
    del_w_1_avg/=10
    del_w_2_avg/=10
    del_bias_1_avg/=10
    del_bias_2_avg/=10
    avg_cost/=10
    #print(avg_cost)

    weights_1 = np.subtract(weights_1, learnrate*del_w_1_avg)
    weights_2 = np.subtract(weights_2, learnrate*del_w_2_avg)
    biases_1 = np.subtract(biases_1, learnrate*del_bias_1_avg)
    biases_2 = np.subtract(biases_2, learnrate*del_bias_2_avg)

learn()

Function to train the model. It is based on stochastic gradient descent. The weights and biases are updated on the basis of the learning rate.

In [21]:
learnrate = 2.0 
for i in range(30):
  learn()

In [22]:
learnrate = 0.4
for i in range(30):
  learn()

Train the model.

In [23]:
feed(test_X[0].reshape(784)/256)

In [24]:
print(outputlayer)

[7.96247261e-04 6.45458261e-02 3.87955048e-03 2.12698218e-02
 1.18225791e-02 8.82553032e-02 3.80486204e-03 8.61885416e-01
 2.73009339e-02 2.94052861e-01]


Just checking to see if the model is able to predict for one value.

In [25]:
print(outputlayer.argmax())

7


In [26]:
print(test_y[0])

7


In [27]:
test_X_upd = np.zeros(784*test_X.shape[0]).reshape(test_X.shape[0], 784)
for i in range(test_X.shape[0]):
  test_X_upd[i] = test_X[i].reshape(784)
  test_X_upd[i] = test_X_upd[i]/256

print(type(test_X_upd[1]))
print(test_X_upd[1].size)

<class 'numpy.ndarray'>
784


Process the test set similar to the training set.

In [28]:
learnrate = 0.01
for i in range(5):
  learn()

Train finally to fine tune.

In [29]:
count = 0
for i in range(60000):
  feed(train_X_upd[i])
  if(outputlayer.argmax() == train_y[i].argmax()):
    count+=1

print(count)

35582


Number of samples correctly predicted in the training set.

In [30]:
count = 0
for i in range(10000):
  feed(test_X_upd[i])
  if(outputlayer.argmax() == test_y[i]):
    count+=1

print(count)

6034


Number of samples correctly predicted in the test set.

In [32]:
accuracy = (count/test_X_upd.shape[0])*100
accuracy

60.34

Calculate the accuracy.